In [1]:
import numpy as np
import scipy as sc

from src.CPQAOA import CP_QAOA
from src.QAOA import QAOA
from src.Pennylane_QAOA import Pennylane_QAOA
#from src.Bluequbit_QAOA import Bluequbit_QAOA
from src.Cirq_QAOA import Cirq_QAOA
from src.Chain import Chain
from src.Grid import Grid
from src.Tools import (portfolio_metrics, 
                       min_cost_partition, 
                       get_qubo, 
                       normalized_cost, 
                       qubo_limits, 
                       check_qubo, 
                       get_ising)
from src.Tools import get_qiskit_H

In [2]:
n_qubits = 11
init_strat = np.array([1 if i%2 == 1 else 0 for i in range(n_qubits)])
k = n_qubits // 2
my_indices = [(i, i+1) for i in range(n_qubits-1)]
n_layers = 2
alpha = 0.5
seed = 0

max_iter = 1500

In [3]:
 # Defining topology
my_chain = Chain(N_qubits=n_qubits)
my_chain.set_initialization_strategy(strategy=init_strat)

# Deciding between grid and 1d chain topology
my_topology = my_chain

# Generating random problem instance 
expected_returns, covariances = portfolio_metrics(n=n_qubits, seed=seed)

# Retrieving C_min, C_max and corresponding states for original portfolio problem
constrained_result, full_result, lmbda = min_cost_partition(nr_qubits=n_qubits,
                                                            k=k,
                                                            mu=expected_returns,
                                                            sigma=covariances,
                                                            alpha=alpha)

portfolio_subspace_max_cost, portfolio_subspace_min_cost, portfolio_subspace_min_state = constrained_result['c_max'], constrained_result['c_min'], constrained_result['s']
#full_space_max_cost = full_result['c_max']
portfolio_subspace_min_state_str = ''.join([str(_) for _ in portfolio_subspace_min_state])

# Generating QUBO corresponding to current problem instance
Q, offset = get_qubo(mu=expected_returns,
                     sigma=covariances, 
                     alpha=alpha,
                     lmbda=lmbda+1e-8, # Adding small constant purposely
                     k=k)
QUBO_limits = qubo_limits(Q=Q,offset=offset)
qubo_min_cost, qubo_max_cost = QUBO_limits['c_min'], QUBO_limits['c_max']
qubo_min_state, qubo_max_state = QUBO_limits['min_state'], QUBO_limits['max_state']
check_qubo(QUBO_matrix=Q, QUBO_offset=offset, expected_returns=expected_returns, covariances=covariances, alpha=alpha, k=k)
qubo_min_state_str = ''.join([str(_) for _ in qubo_min_state])


if not portfolio_subspace_min_state_str == qubo_min_state_str:
    raise RuntimeError(f'portfolio_subspace_min_state_str: {portfolio_subspace_min_state_str}, qubo_min_state_str={qubo_min_state_str}'+f'Min. cost of qubo is: {qubo_min_cost}, but min. cost of constrained portfolio is: {portfolio_subspace_min_cost}.')

if not np.isclose(qubo_min_cost,portfolio_subspace_min_cost):
    raise RuntimeError(f'Min. cost of qubo is: {qubo_min_cost}, but min. cost of constrained portfolio is: {portfolio_subspace_min_cost}.')

if not qubo_max_cost >= portfolio_subspace_max_cost:
    raise RuntimeError(f'Max. cost of qubo: {qubo_max_cost}, max. cost of portfolio subspace: {portfolio_subspace_max_cost} (should be qubo max. >= constrained portfolio max)')

In [6]:
qiskit_ansatz = QAOA(N_qubits=n_qubits,
                     layers=n_layers,
                     QUBO_matrix=Q,
                     QUBO_offset=offset,
                     constraining_mixer=False,
                     Topology=my_topology)

pennylane_ansatz = Pennylane_QAOA(N_qubits=n_qubits,
                                  layers=n_layers,
                                  QUBO_matrix=Q,
                                  QUBO_offset=offset)

"""bluequbit_ansatz = Bluequbit_QAOA(N_qubits=n_qubits,
                                  layers=n_layers,
                                  QUBO_matrix=Q,
                                  QUBO_offset=offset)"""

cirq_ansatz = Cirq_QAOA(N_qubits=n_qubits,
                                  layers=n_layers,
                                  QUBO_matrix=Q,
                                  QUBO_offset=offset)

In [7]:
# Choosing optimizer for scipy
available_methods = ['COBYLA', 'Nelder-Mead', 'BFGS']
optimizer_method = available_methods[0]

# Generating callback function for plotting
qiskit_costs = [] # Normalized costs
qiskit_probs = [] # probability of optimal state
def qiskit_callback(x):
    _dict_ = qiskit_ansatz.get_state_probabilities(flip_states=False)
    # N.B. Normalizing w. respect to full space max cost
    _cost_ = normalized_cost(result=_dict_,
                             QUBO_matrix=Q,
                             QUBO_offset=offset,
                             max_cost=portfolio_subspace_max_cost, 
                             min_cost=qubo_min_cost)
    if portfolio_subspace_min_state_str in list(_dict_.keys()):
        qiskit_probs.append(_dict_[portfolio_subspace_min_state_str])
    else:
        qiskit_probs.append(0)
    qiskit_costs.append(_cost_)
    
# Generating callback function for plotting
pennylane_costs = [] # Normalized costs
pennylane_probs = [] # probability of optimal state
def pennylane_callback(x):
    _dict_ = pennylane_ansatz.get_state_probabilities(flip_states=False)
    # N.B. Normalizing w. respect to full space max cost
    _cost_ = normalized_cost(result=_dict_,
                             QUBO_matrix=Q,
                             QUBO_offset=offset,
                             max_cost=portfolio_subspace_max_cost, 
                             min_cost=qubo_min_cost)
    if portfolio_subspace_min_state_str in list(_dict_.keys()):
        pennylane_probs.append(_dict_[portfolio_subspace_min_state_str])
    else:
        pennylane_probs.append(0)
    pennylane_costs.append(_cost_)
    

"""# Generating callback function for plotting
bluequbit_costs = [] # Normalized costs
bluequbit_probs = [] # probability of optimal state
def bluequbit_callback(x):
    _dict_ = bluequbit_ansatz.get_state_probabilities(flip_states=False)
    # N.B. Normalizing w. respect to full space max cost
    _cost_ = normalized_cost(result=_dict_,
                             QUBO_matrix=Q,
                             QUBO_offset=offset,
                             max_cost=portfolio_subspace_max_cost, 
                             min_cost=qubo_min_cost)
    if portfolio_subspace_min_state_str in list(_dict_.keys()):
        bluequbit_probs.append(_dict_[portfolio_subspace_min_state_str])
    else:
        bluequbit_probs.append(0)
    bluequbit_costs.append(_cost_)"""


# Generating callback function for plotting
cirq_costs = [] # Normalized costs
cirq_probs = [] # probability of optimal state
def cirq_callback(x):
    _dict_ = cirq_ansatz.get_state_probabilities(flip_states=False)
    # N.B. Normalizing w. respect to full space max cost
    _cost_ = normalized_cost(result=_dict_,
                             QUBO_matrix=Q,
                             QUBO_offset=offset,
                             max_cost=portfolio_subspace_max_cost, 
                             min_cost=qubo_min_cost)
    if portfolio_subspace_min_state_str in list(_dict_.keys()):
        cirq_probs.append(_dict_[portfolio_subspace_min_state_str])
    else:
        cirq_probs.append(0)
    cirq_costs.append(_cost_)

# Generating initial guess for rotation angles
np.random.seed(seed)
theta_min, theta_max = -2*np.pi, 2*np.pi
N_angles = 2 * n_layers
theta_i = np.random.uniform(low=theta_min, high=theta_max, size=N_angles)

In [8]:
qiskit_res = sc.optimize.minimize(fun=qiskit_ansatz.get_cost, 
                                  x0=theta_i,
                                  method=optimizer_method,
                                  options={'disp': False, 
                                           'maxiter': max_iter},
                                  callback=qiskit_callback)
qiskit_res


 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -0.20516798999500008
       x: [ 3.974e-01  2.833e+00  1.916e+00  1.485e+00]
    nfev: 595
   maxcv: 0.0

In [9]:
pennylane_res = sc.optimize.minimize(fun=pennylane_ansatz.get_cost, 
                                     x0=theta_i,
                                     method=optimizer_method,
                                     options={'disp': False, 
                                               'maxiter': max_iter},
                                     callback=pennylane_callback)
pennylane_res

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -0.20323057649361975
       x: [ 7.483e-01  2.713e+00  2.240e+00  1.333e+00]
    nfev: 425
   maxcv: 0.0

In [10]:
"""bluequbit_res = sc.optimize.minimize(fun=bluequbit_ansatz.get_cost, 
                                     x0=theta_i,
                                     method=optimizer_method,
                                     options={'disp': False, 
                                               'maxiter': max_iter},
                                     callback=bluequbit_callback)
bluequbit_res"""

"bluequbit_res = sc.optimize.minimize(fun=bluequbit_ansatz.get_cost, \n                                     x0=theta_i,\n                                     method=optimizer_method,\n                                     options={'disp': False, \n                                               'maxiter': max_iter},\n                                     callback=bluequbit_callback)\nbluequbit_res"

In [11]:
cirq_res = sc.optimize.minimize(fun=cirq_ansatz.get_cost, 
                                     x0=theta_i,
                                     method=optimizer_method,
                                     options={'disp': False, 
                                               'maxiter': max_iter},
                                     callback=cirq_callback)
cirq_res

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -0.20030032497387046
       x: [ 2.059e+00  2.686e+00  9.315e-01  5.305e-01]
    nfev: 60
   maxcv: 0.0

In [12]:
qiskit_ansatz.circuit_time, qiskit_ansatz.cost_time

(16.30989980697632, 3.0329692363739014)

In [13]:
pennylane_ansatz.circuit_time, pennylane_ansatz.cost_time

(8.38973093032837, 2.060220241546631)

In [14]:
#bluequbit_ansatz.circuit_time, bluequbit_ansatz.cost_time

In [15]:
cirq_ansatz.circuit_time, cirq_ansatz.cost_time

(2.0803475379943848, 0.28264951705932617)